In [1]:
import numpy as np 
import pandas as pd 
import nltk
from nltk.corpus import stopwords
import gensim
import string

In [2]:
df = pd.read_csv('imdb_sentiment.csv')

### Load Google's Pretrained model
- Download: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
- Size: 1.5GB

In [5]:
google_model = '..//word2vec//GoogleNews-vectors-negative300.bin.gz' 
embeddings = gensim.models.KeyedVectors.load_word2vec_format(google_model,
                                                        binary=True)

In [6]:
embeddings['google']

array([ 0.11279297, -0.22949219,  0.01782227,  0.19238281, -0.22558594,
        0.00933838, -0.01708984, -0.17285156,  0.1953125 , -0.05444336,
        0.13867188, -0.27148438, -0.26757812,  0.08056641, -0.21191406,
        0.31835938,  0.10693359, -0.00732422,  0.02758789, -0.40234375,
       -0.07763672,  0.2421875 ,  0.09912109,  0.06689453, -0.36328125,
       -0.09521484, -0.09423828,  0.11474609, -0.07128906, -0.14355469,
       -0.10302734,  0.11376953, -0.22070312,  0.01489258, -0.125     ,
        0.05566406, -0.07080078,  0.14648438,  0.32226562,  0.21679688,
        0.12207031, -0.09863281,  0.05566406,  0.31445312,  0.20605469,
       -0.0246582 ,  0.07226562, -0.25390625, -0.13085938, -0.20800781,
       -0.22851562,  0.0189209 , -0.06787109, -0.00218201, -0.0112915 ,
        0.04101562, -0.04907227, -0.10791016,  0.06591797, -0.02526855,
        0.046875  , -0.27148438, -0.13476562,  0.08544922, -0.30078125,
       -0.09375   , -0.36523438, -0.17871094, -0.07666016, -0.02

In [54]:
embeddings['google'].shape

(300,)

In [55]:
df.head(5)

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [27]:
df.shape

(5728, 2)

In [57]:
df.columns

Index(['review', 'sentiment'], dtype='object')

In [58]:
df.drop_duplicates(inplace = True)

In [59]:
df.shape

(745, 2)

In [60]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [61]:
from string import punctuation
from nltk.corpus import stopwords
from nltk import word_tokenize

In [62]:
def clean_txt(sent):
    tokens = word_tokenize(sent.lower())
    sent.replace('[^a-z ]',' ')
    stop_updated = stopwords.words("english") + list(punctuation) + ["..."] + \
    ["would", "could","told","subject"]
    text = [term for term in tokens if term not in stop_updated and len(term) > 2] 
    res = " ".join(text)
    return res

In [64]:
df['clean_review'] = df.review.apply(clean_txt)

In [65]:
df.head()

,review,sentiment,clean_review
0,"A very, very, very slow-moving, aimless movie ...",0,slow-moving aimless movie distressed drifting ...
1,Not sure who was more lost - the flat characte...,0,sure lost flat characters audience nearly half...
2,Attempting artiness with black & white and cle...,0,attempting artiness black white clever camera ...
3,Very little music or anything to speak of.,0,little music anything speak
4,The best scene in the movie was when Gerardo i...,1,best scene movie gerardo trying find song keep...


## Create document vectors using word embeddings

In [66]:
docs_vectors = pd.DataFrame()
for doc in df.clean_review:
    temp = pd.DataFrame()
    words = doc.split(' ')
    for word in words:
        try:
            word2vec = embeddings[word]
            temp = temp.append(pd.Series(word2vec),ignore_index=True)
        except:
            pass
    doc_vector = temp.mean()
    docs_vectors = docs_vectors.append(doc_vector,ignore_index=True)
docs_vectors.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.170980,0.169840,-0.111511,0.045278,-0.063273,0.021952,-0.062159,-0.018341,0.048421,0.052041,...,-0.073405,0.086751,-0.090210,-0.075134,-0.072550,0.062012,0.072103,-0.005147,0.044312,0.040568
1,0.143494,-0.017984,-0.009964,0.106308,-0.052292,-0.075745,-0.015942,-0.037323,0.114197,0.029335,...,-0.080658,0.051147,-0.152344,-0.048691,-0.065536,-0.029953,0.032959,-0.053650,0.021364,-0.096176
2,0.119808,0.053776,-0.008872,0.010807,-0.072973,-0.016860,0.056718,0.027276,0.051528,0.033042,...,-0.117338,0.073895,-0.129391,0.028119,-0.094421,-0.022527,0.060147,-0.040308,0.022281,0.013235
3,0.070648,-0.059143,-0.003357,0.084737,-0.092316,0.107788,0.156372,-0.170471,0.006287,-0.007477,...,-0.052734,0.080322,-0.023529,0.030060,-0.011902,-0.059326,0.144897,-0.071167,0.074524,-0.052612
4,0.120809,0.007568,0.040127,0.048381,-0.056264,0.004886,0.024143,-0.053096,0.066806,-0.012214,...,-0.098731,0.089857,-0.140744,0.013428,-0.042019,-0.015290,0.079617,0.000027,0.037869,0.076067


In [67]:
docs_vectors.isna().sum().head().head()

0    2
1    2
2    2
3    2
4    2
dtype: int64

In [68]:
docs_vectors = docs_vectors.fillna(method='ffill')

In [69]:
X = docs_vectors
y = df.sentiment.values

In [145]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=100)

we cannot use naive bayes with word embeddings. 
word embeddings can be either +ve or -ve and naive bayes cannot take negetive values

In [114]:
from sklearn.tree import DecisionTreeClassifier 
classifier = DecisionTreeClassifier(criterion="entropy", max_depth=5, random_state=100)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

In [154]:
from sklearn.linear_model import LogisticRegression 
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=150, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [155]:
y_pred = classifier.predict(X_test)

In [152]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [156]:
accuracy_score(y_test,y_pred)

0.8526785714285714

In [128]:
confusion_matrix(y_test,y_pred)

array([[67, 12],
       [11, 59]], dtype=int64)